In [1]:
!pip install ultralytics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.6/599.6 kB 16.2 MB/s eta 0:00:00


In [2]:
import cv2
import os
import shutil
import glob
import ultralytics
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#Download YOLO v5
#pip install -r yolov5/requirements.txt
#! pip install scipy --upgrade
# !git clone https://github.com/ultralytics/yolov5  # clone
# %cd yolov5
# !pip install -r requirements.txt  # install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
path = '/content/drive/MyDrive/mAP_data'
#/content/drive/MyDrive/mAP_data/train

In [6]:
def boxes_image(image_path, annotation_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    with open(annotation_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        values = list(map(float, line.strip().split(' ')))
        class_id = int(values[0])
        x_min, y_min = int(round(values[1])), int(round(values[2]))
        x_max, y_max = int(round(max(values[3], values[5], values[7]))), int(round(max(values[4], values[6], values[8])))

        #Draw Box

        cv2.rectangle(image,( x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
        cv2.putText(image, str(class_id), (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    plt.figure(figsize=(25, 25))
    plt.imshow(image)
    plt.show()



In [7]:
def normalize_texts(texts_list):
    for file in tqdm(texts_list):
        file_name = os.path.basename(file)
        file_name = file_name.split('.')[0]
        img = cv2.imread(f'/content/drive/MyDrive/mAP_data/train/{file_name}.png')
        img_height, img_width, _ = img.shape
        cv2.destroyAllWindows()
        with open(file, 'r') as f:
            lines_list = []
            lines = f.readlines()
            for line in lines:
                values = list(map(float, line.strip().split(' ')))
                class_id = int(values[0])
                x_min, y_min = float(min(values[5], values[7])), float(min(values[6], values[8]))
                x_max, y_max = float(max(values[1], values[3])), float(max(values[2], values[4]))
                x_center, y_center = float(((x_min + x_max) / 2) / img_width), float(((y_min + y_max) / 2) / img_height)
                width, height = abs(x_max - x_min) / img_width, abs(y_max - y_min) / img_height
                lines_list.append([class_id, x_center, y_center, width, height])
        with open(f'/content/drive/MyDrive/mAP_data/label/{file_name}.txt', 'w') as f:
            for line in lines_list:
                f.write(str(line[0]) + ' ' + str(line[1]) + ' ' + str(line[2]) + ' ' + str(line[3]) + ' ' + str(line[4]) + '\n')
    print('finish')

In [11]:
#checking label_files
if not os.path.exists('/content/drive/MyDrive/mAP_data/label'):
    os.makedirs('/content/drive/MyDrive/mAP_data/label')

if (len(os.listdir('/content/drive/MyDrive/mAP_data/label'))) != 6482:
    print('start moving')
    #원상태
    path = '/content/drive/MyDrive/mAP_data/'
    train_folder = f'{path}train'
    valid_folder = f'{path}valid'
    #test_folder = f'{path}yolo/test'

    train_images = sorted(glob.glob(f'{path}/yolo/train/*.png'))
    train_txts = sorted(glob.glob(f'{path}/yolo/train/*.txt'))

    valid_images = sorted(glob.glob(f'{path}/yolo/valid/*.png'))
    valid_txts = sorted(glob.glob(f'{path}/yolo/valid/*.txt'))

    for train_image in tqdm(train_images, desc='Copying train images'):
        shutil.move(train_image, train_folder)
    for train_txt in tqdm(train_txts, desc='Copying train annotations'):
        try:
            shutil.move(train_txt, train_folder)
        except:
            pass
    for valid_image in tqdm(valid_images, desc='Copying valid images'):
        shutil.move(valid_image, train_folder)
    for valid_txt in tqdm(valid_txts, desc='Copying valid annotations'):
        try:
            shutil.move(valid_txt, train_folder)
        except:
            pass
    #normalize_texts
    %cd /content/drive/MyDrive/mAP_data/label/
    !unzip "label.zip"
    %cd /content
else:
    print('skip!')

skip!


In [12]:
#test path
image_path = f'{path}/train/syn_00301.png'
annotation_path = f'{path}/train/syn_00301.txt'

In [13]:
#boxes_image(image_path, annotation_path)

In [14]:
#Checking dir & install YOLO
#path = /content/drive/MyDrive/mAP_data/
if os.path.exists("/content/drive/MyDrive/mAP_data/yolo/yolov5"):
    shutil.rmtree('/content/drive/MyDrive/mAP_data/yolo/yolov5')

%cd /content/drive/MyDrive/mAP_data/yolo/
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
!pip install -r requirements.txt  # install

if not os.path.exists('/content/drive/MyDrive/mAP_data/yolo/train'):
    os.makedirs('/content/drive/MyDrive/mAP_data/yolo/train')

if not os.path.exists('/content/drive/MyDrive/mAP_data/yolo/valid'):
    os.makedirs('/content/drive/MyDrive/mAP_data/yolo/valid')

if not os.path.exists('/content/drive/MyDrive/mAP_data/yolo/test'):
    os.makedirs('/content/drive/MyDrive/mAP_data/yolo/test')

if not os.path.exists('/content/drive/MyDrive/mAP_data/results'):
    os.makedirs('/content/drive/MyDrive/mAP_data/results')

/content/drive/MyDrive/mAP_data/yolo
Cloning into 'yolov5'...
remote: Enumerating objects: 15973, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 15973 (delta 84), reused 97 (delta 63), pack-reused 15831
Receiving objects: 100% (15973/15973), 14.57 MiB | 14.21 MiB/s, done.
Resolving deltas: 100% (10958/10958), done.
/content/drive/MyDrive/mAP_data/yolo/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.5 MB/s eta 0:00:00


In [15]:
path = '/content/drive/MyDrive/mAP_data/'
train_folder = f'{path}train'
valid_folder = f'{path}valid'
#test_folder = f'{path}yolo/test'
train_images = sorted(glob.glob(f'{path}/yolo/train/*.png'))
train_txts = sorted(glob.glob(f'{path}/yolo/train/*.txt'))
valid_images = sorted(glob.glob(f'{path}/yolo/valid/*.png'))
valid_txts = sorted(glob.glob(f'{path}/yolo/valid/*.txt'))
for train_image in tqdm(train_images, desc='Copying train images'):
    shutil.move(train_image, train_folder)
for train_txt in tqdm(train_txts, desc='Copying train annotations'):
    try:
        shutil.move(train_txt, train_folder)
    except:
        pass
for valid_image in tqdm(valid_images, desc='Copying valid images'):
    shutil.move(valid_image, train_folder)
for valid_txt in tqdm(valid_txts, desc='Copying valid annotations'):
    try:
        shutil.move(valid_txt, train_folder)
    except:
        pass

Copying valid annotations: 100%|██████████| 973/973 [00:00<00:00, 1831.63it/s]


In [18]:
#divide train, validation datasets
path = '/content/drive/MyDrive/mAP_data/'
image_paths = sorted(glob.glob(f'{path}/train/*.png'))
text_paths = sorted(glob.glob(f'{path}/label/*.txt'))
print(len(image_paths), len(text_paths))
train_images, valid_images, train_txts, valid_txts = train_test_split(image_paths, text_paths, test_size=0.1, random_state=69)

6481 6481


In [19]:
print(len(image_paths), len(text_paths))

6481 6481


In [20]:
#folder
train_folder = f'{path}yolo/train'
valid_folder = f'{path}yolo/valid'
test_folder = f'{path}yolo/test'

train_images.sort()
valid_images.sort()
train_txts.sort()
valid_txts.sort()

for train_image in tqdm(train_images, desc='Copying train images'):
    shutil.move(train_image, train_folder)
for train_txt in tqdm(train_txts, desc='Copying train annotations'):
    try:
        shutil.move(train_txt, train_folder)
    except:
        pass

for valid_image in tqdm(valid_images, desc='Copying valid images'):
    shutil.move(valid_image, valid_folder)
for valid_txt in tqdm(valid_txts, desc='Copying valid annotations'):
    try:
        shutil.move(valid_txt, train_folder)
    except:
        pass

print(len(train_folder),len(valid_folder), len(test_folder))

Copying valid annotations: 100%|██████████| 649/649 [00:00<00:00, 2202.58it/s]

42 42 41


#Yolo yaml 파일 만들기

In [22]:
#Create yaml file
import yaml
#open classes.txt
with open(f'{path}/classes.txt', 'r') as cl:
    lines = cl.readlines()

    classes = [line.strip().split(',')[1] for line in lines]
data_yaml = {
    'nc': len(classes),
    'names' : classes,
    'train': '/content/drive/MyDrive/mAP_data/yolo/train',
    'val': '/content/drive/MyDrive/mAP_data/yolo/valid',
    'test': '/content/drive/MyDrive/mAP_data/test'
}
with open(f'{path}/data.yaml', 'w')as f:
    yaml.dump(data_yaml, f)

In [23]:
test = '000,chevrolet_malibu_sedan_2012_2016'
print(test.strip().split(',')[1])
#[line.strip().split(',')[1] for line in lines]

chevrolet_malibu_sedan_2012_2016


In [24]:
!cat /content/drive/MyDrive/mAP_data/data.yaml

names:
- chevrolet_malibu_sedan_2012_2016
- chevrolet_malibu_sedan_2017_2019
- chevrolet_spark_hatchback_2016_2021
- chevrolet_trailblazer_suv_2021_
- chevrolet_trax_suv_2017_2019
- genesis_g80_sedan_2016_2020
- genesis_g80_sedan_2021_
- genesis_gv80_suv_2020_
- hyundai_avante_sedan_2011_2015
- hyundai_avante_sedan_2020_
- hyundai_grandeur_sedan_2011_2016
- hyundai_grandstarex_van_2018_2020
- hyundai_ioniq_hatchback_2016_2019
- hyundai_sonata_sedan_2004_2009
- hyundai_sonata_sedan_2010_2014
- hyundai_sonata_sedan_2019_2020
- kia_carnival_van_2015_2020
- kia_carnival_van_2021_
- kia_k5_sedan_2010_2015
- kia_k5_sedan_2020_
- kia_k7_sedan_2016_2020
- kia_mohave_suv_2020_
- kia_morning_hatchback_2004_2010
- kia_morning_hatchback_2011_2016
- kia_ray_hatchback_2012_2017
- kia_sorrento_suv_2015_2019
- kia_sorrento_suv_2020_
- kia_soul_suv_2014_2018
- kia_sportage_suv_2016_2020
- kia_stonic_suv_2017_2019
- renault_sm3_sedan_2015_2018
- renault_xm3_suv_2020_
- ssangyong_korando_suv_2019_2020
- 

In [25]:
%cd /content/drive/MyDrive/mAP_data/yolo/yolov5
!pwd

/content/drive/MyDrive/mAP_data/yolo/yolov5
/content/drive/MyDrive/mAP_data/yolo/yolov5


###Yolov5

In [27]:
#YOLOv5m
!python train.py --img 960 --epochs 20 --batch 32 --save-period 10 --cache --data /content/drive/MyDrive/mAP_data/data.yaml --weights  yolov5s.pt --device 0 --optimizer Adam --project /content/drive/MyDrive/mAP_data/yolo/results --name exp

train: weights=yolov5s.pt, cfg=, data=/content/drive/MyDrive/mAP_data/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=20, batch_size=32, imgsz=960, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=0, multi_scale=False, single_cls=False, optimizer=Adam, sync_bn=False, workers=8, project=/content/drive/MyDrive/mAP_data/yolo/results, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=10, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-181-g3812a1a Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2

### Yolov8

In [28]:
#YOLOv8  path : /content/drive/MyDrive/mAP_data/yolo/results/exp3/weights/last.pt
#!yolo train model=/content/drive/MyDrive/mAP_data/yolo/results/exp5/weights/last.pt data=/content/drive/MyDrive/mAP_data/data.yaml epochs=10 imgsz=640 batch=32 save=True save_period=5 cache=True device=0 project=/content/drive/MyDrive/mAP_data/yolo/results name=exp optimizer=AdamW verbose=True resume=False workers=16 val=True

In [ ]:
# #model = ultralytics.YOLO('/content/drive/MyDrive/mAP_data/yolo/results/exp25/weights/best.pt')

# model = ultralytics.YOLO('/content/drive/MyDrive/mAP_data/yolo/results/exp74/weights/last.pt')

In [30]:
# model = ultralytics.YOLO('yolov8m')
# #model = ultralytics.YOLO('yolov8m').load('/content/drive/MyDrive/mAP_data/yolo/results/exp25/weights/best.pt')
# results = model.train(
#     #model='yolov8m',
#     data='/content/drive/MyDrive/mAP_data/data.yaml',
#     imgsz = 960,
#     epochs=200,
#     batch=16,
#     patience=200,
#     warmup_epochs=0,
#     #workers=16,
#     lr0=0.01,
#     #lrf=0.0001,
#     device=0,
#     project='/content/drive/MyDrive/mAP_data/yolo/results',
#     name='exp',
#     seed=69,
#     optimizer='Adam',
#     val=True,
#     cache=False,
#     pretrained=True,
#     save_period=10,
#     resume=True
#     #weight_decay=0.00005
# )


In [ ]:
BATCH_SIZE=34
def get_test_image_paths(test_image_paths):
    for i in range(0, len(test_image_paths), BATCH_SIZE):
        yield test_image_paths[i:i+BATCH_SIZE]

In [ ]:
test_image_paths = glob.glob(f'/content/drive/MyDrive/mAP_data/test/*.png')
(test_image_paths)

In [ ]:
model = ('yolov8m')
model = ultralytics.YOLO('/content/drive/MyDrive/mAP_data/yolo/results/exp62/weights/best.pt')
for i, image in tqdm(enumerate(get_test_image_paths(test_image_paths)), total=int(len(test_image_paths)/BATCH_SIZE)):
    print(i, len(image))
    model.predict(source=image, imgsz=960, iou=0.2, conf=0.1, save_conf=True, save_txt=True, project='/content/drive/MyDrive/mAP_data/results', name='pred', exist_ok=False, device=0, verbose=True, augment=True)

In [ ]:
len(glob.glob('/content/drive/MyDrive/mAP_data/results/pred24/labels/*.txt'))

In [ ]:
def yolo_to_labelme(line, image_width, image_height, txt_file_name):
    file_name = txt_file_name.split("/")[-1].replace(".txt", ".png")
    class_id, x, y, width, height, confidence = [float(temp) for temp in line.split()]

    x_min = int((x - width / 2) * image_width)
    x_max = int((x + width / 2) * image_width)
    y_min = int((y - height / 2) * image_height)
    y_max = int((y + height / 2) * image_height)

    return file_name, int(class_id), confidence, x_min, y_min, x_max, y_min, x_max, y_max, x_min, y_max

In [ ]:
import pandas as pd

In [ ]:
infer_txts = glob.glob('/content/drive/MyDrive/mAP_data/results/pred24/labels/*.txt')

results = []
for infer_txt in tqdm(infer_txts):
    base_file_name = infer_txt.split("/")[-1].split(".")[0]
    imgage_height, imgage_width = cv2.imread(f"/content/drive/MyDrive/mAP_data/test/{base_file_name}.png").shape[:2]
    with open(infer_txt, "r") as reader:
        lines = reader.readlines()
        for line in lines:
            results.append(yolo_to_labelme(line, imgage_width, imgage_height, infer_txt))

df_submission = pd.DataFrame(data=results, columns=["file_name", "class_id", "confidence", "point1_x", "point1_y", "point2_x", "point2_y", "point3_x", "point3_y", "point4_x", "point4_y"])
df_submission.to_csv(f"/content/result.csv", index=False)